In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
import torch
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int
from muutils.dictmagic import condense_tensor_dict

In [52]:
import tkinter as tk
from tkinter import simpledialog, messagebox
from transformers import pipeline
import csv
import os  # Ensure os is imported at the top

def load_model(model_name):
    """ Load a model as a text generation pipeline. """
    return pipeline("text-generation", model=model_name)

class App:
    def __init__(self, root):
        self.root = root
        root.title("Model Comparison Tool")

        self.model1 = load_model("gpt2")  
        self.model2 = load_model('EleutherAI/gpt-neo-1.3B')  

        # Setup the UI
        tk.Label(root, text="Enter your prompt:").pack()
        
        self.prompt_entry = tk.Entry(root, width=50)
        self.prompt_entry.pack()

        self.generate_button = tk.Button(root, text="Generate Texts", command=self.generate_texts)
        self.generate_button.pack()

        self.output1_label = tk.Label(root, text="", wraplength=300)
        self.output1_label.pack()

        self.output2_label = tk.Label(root, text="", wraplength=300)
        self.output2_label.pack()

        self.select_button1 = tk.Button(root, text="Select Output 1", command=lambda: self.update_model(1))
        self.select_button1.pack()
        self.select_button1.config(state="disabled")  # Initially disabled

        self.select_button2 = tk.Button(root, text="Select Output 2", command=lambda: self.update_model(2))
        self.select_button2.pack()
        self.select_button2.config(state="disabled")  # Initially disabled

    def generate_texts(self):
        prompt = self.prompt_entry.get()
        result1 = self.model1(prompt, max_length=50, truncation=True)
        result2 = self.model2(prompt, max_length=50, truncation=True)
        self.output1 = result1[0]['generated_text']
        self.output2 = result2[0]['generated_text']

        self.output1_label.config(text=self.output1)
        self.output2_label.config(text=self.output2)

        self.select_button1.config(state="normal")
        self.select_button2.config(state="normal")
        
    def log_user_feedback(self, prompt, selected_output, model_name):
        # Check if the file exists to determine if headers are needed
        file_exists = os.path.isfile('user_feedback.csv')
        
        with open('user_feedback.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            # Write headers if the file is being created
            if not file_exists:
                writer.writerow(['prompt', 'selected_output', 'model_name'])
            
            # Write the actual data
            writer.writerow([prompt, selected_output, model_name])

    def update_model(self, selected_model):
        prompt = self.prompt_entry.get()
        selected_output = self.output1 if selected_model == 1 else self.output2
        model_name = "model1" if selected_model == 1 else "model2"
        
        self.log_user_feedback(prompt, selected_output, model_name)
        messagebox.showinfo("Selection", "Feedback recorded! Generating new texts...")
        self.generate_texts()

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [55]:

from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

def fine_tune_model(model_name, csv_file, output_dir):
    # Load tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    
    # Set padding token if undefined
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Make sure the model's embedding sizes match if you're changing the tokenizer's pad token
    model.config.pad_token_id = tokenizer.pad_token_id

    # Load dataset
    dataset = load_dataset('csv', data_files=csv_file, split='train')
    
    # Define preprocessing function to concatenate prompt and selected_output
    def preprocess_function(examples):
        texts = [p + " " + o for p, o in zip(examples['prompt'], examples['selected_output'])]
        return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

    # Map preprocessing function
    dataset = dataset.map(preprocess_function, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    # Data collator for padding
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
    )

    # Initialize and run trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()

    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

# Call function to fine-tune model
fine_tune_model('gpt2', 'user_feedback.csv', 'fine_tuned_model')

# API KEY 5f9ac9efd76e998d6fc84ca0527f4e5123ec495b


Map: 100%|██████████| 3/3 [00:00<00:00, 171.84 examples/s]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Geoff McIntyre\.netrc


100%|██████████| 3/3 [00:06<00:00,  2.25s/it]


{'train_runtime': 84.5422, 'train_samples_per_second': 0.106, 'train_steps_per_second': 0.035, 'train_loss': 2.884645462036133, 'epoch': 3.0}
